In [2]:
import numpy as np
import pandas as pd
import pickle
import datetime
%matplotlib inline

## Update Local Path

In [6]:
#path = "../../../Google Drive/Data_science/NYU/Machine Learning/ML Project (Collisions)/" #Joe
path = "../../../../Google Drive/ML Project (Collisions)/" # Joyce
# path = "" # Lucas

In [ ]:
df = pd.read_csv(path + "NYPD_Motor_Vehicle_Collisions.csv", parse_dates=[['DATE', 'TIME']], infer_datetime_format=True)
df = df[df.LATITUDE.notnull()]
df = df.reset_index()
to_bin = lambda x: np.floor(x / step) * step
df["lat_bin"] = df.LATITUDE.map(to_bin)
df["long_bin"] = df.LONGITUDE.map(to_bin)

# Modify these parameters!

In [ ]:
#Bin size
step = 0.005
#Number of days before time to count complaints
days = 90

## Join 311 data

In [6]:
complaints = pd.read_csv(path + "Google_Map_-_311_NYPD_Traffic_Complaints_-_2010-Present.csv")
complaints["lat_bin"] = complaints.Latitude.map(to_bin)
complaints["long_bin"] = complaints.Longitude.map(to_bin)

In [7]:
grouped = pd.DataFrame(complaints.groupby(['lat_bin', 'long_bin', 'Descriptor'])['Unique Key'].count().reset_index(level=2))

In [8]:
for comp_type in grouped.Descriptor.unique():
    renamed = grouped[grouped['Descriptor'] == comp_type].rename(columns={'Unique Key': comp_type+' total bin'}).drop(['Descriptor'], axis=1)
    df = df.join(renamed, on=['lat_bin', 'long_bin'], how='left')
    df[comp_type+' total bin'] = df[comp_type+' total bin'].fillna(value=0)

In [9]:
complaints['Created Date'] = pd.to_datetime(complaints['Created Date'], infer_datetime_format=True)

In [10]:
def time_before(lat, lon, start_date, days):
    within = complaints[(complaints['Created Date'] < start_date) & (complaints['Created Date'] > start_date - datetime.timedelta(days))]
    groupedtime = pd.DataFrame(within.groupby(['lat_bin', 'long_bin', 'Descriptor'])['Unique Key'].count().reset_index(level=2))

    counts = []
    
    for comp_type in grouped.Descriptor.unique():
        try:
            value = int(groupedtime[groupedtime['Descriptor'] == comp_type].loc[lat,lon].values[1])
        except:
            value = 0
            
        counts.append(value)    
    return counts

In [19]:
before_counts = []

for idx, row in df.iterrows():
    counts = time_before(row['lat_bin'], row['long_bin'], row['DATE_TIME'], days)
    before_counts.append(counts)
    
    if idx % 100 == 0:
        print '%s/%s rows completed' % (idx, df.shape[0])

0/792851 rows completed
100/792851 rows completed
200/792851 rows completed
300/792851 rows completed
400/792851 rows completed
500/792851 rows completed
600/792851 rows completed
700/792851 rows completed
800/792851 rows completed
900/792851 rows completed
1000/792851 rows completed
1100/792851 rows completed
1200/792851 rows completed
1300/792851 rows completed
1400/792851 rows completed
1500/792851 rows completed
1600/792851 rows completed
1700/792851 rows completed
1800/792851 rows completed
1900/792851 rows completed
2000/792851 rows completed
2100/792851 rows completed
2200/792851 rows completed
2300/792851 rows completed
2400/792851 rows completed
2500/792851 rows completed
2600/792851 rows completed
2700/792851 rows completed
2800/792851 rows completed
2900/792851 rows completed
3000/792851 rows completed
3100/792851 rows completed
3200/792851 rows completed
3300/792851 rows completed
3400/792851 rows completed
3500/792851 rows completed
3600/792851 rows completed
3700/792851 r

In [23]:
j = 0

for comp_type in grouped.Descriptor.unique():
    df[comp_type+' count '+days+' days before'] = [i[j] for i in before_counts]
    j += 1

In [40]:
df_311_joined = df.set_index(['UNIQUE KEY'].df.drop(df.columns.values[:30], axis=1)

In [45]:
with open('311_joined.pkl', 'wb') as output:
    pickle.dump(df_311_joined, output, pickle.HIGHEST_PROTOCOL)